<a href="https://colab.research.google.com/github/TADY25/Unil_Tissot/blob/main/DM_Tissot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:

!pip install -U spacy
!python -m spacy download fr
!python -m spacy download fr_core_news_md

⚠ As of spaCy v3.0, shortcuts like 'fr' are deprecated. Please use the
full pipeline package name 'fr_core_news_sm' instead.
     |████████████████████████████████| 17.4 MB 470 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
     |████████████████████████████████| 46.9 MB 2.6 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_md')


In [8]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd 
import seaborn as sns
sns.set_style("whitegrid")

import spacy
from spacy import displacy
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [ ]:
df=pd.read_csv('training_data.csv')
df.head()


,id,sentence,difficulty
0,0,Les coûts kilométriques réels peuvent diverger...,C1
1,1,"Le bleu, c'est ma couleur préférée mais je n'a...",A1
2,2,Le test de niveau en français est sur le site ...,A1
3,3,Est-ce que ton mari est aussi de Boston?,A1
4,4,"Dans les écoles de commerce, dans les couloirs...",B1


In [10]:
sp = spacy.load('fr')


OSError: ignored